In [1]:
import io
import zipfile
import requests
import pandas as pd
from pathlib import Path

# =========================================================
# WDI URBAN CORE PACK — DOWNLOAD + LIMPEZA
# =========================================================

# Dicionário: código WDI -> nome da coluna que vamos criar
wdi_indicators = {
    "SP.URB.TOTL.IN.ZS":   "wdi_urb_pop_pct",        # % da população que vive em áreas urbanas
    "SP.URB.GROW":         "wdi_urb_pop_growth",     # crescimento anual da população urbana
    "EN.ATM.CO2E.PC":      "wdi_co2_per_capita",     # CO2 per capita
    "EN.ATM.PM25.MC.M3":   "wdi_pm25",               # material particulado fino
    "EG.FEC.RNEW.ZS":      "wdi_renew_energy_pct",   # % energia renovável
    "SH.STA.BASS.UR.ZS":   "wdi_basic_sanitation_urb_pct",  # saneamento básico em área urbana
    "SH.H2O.BASW.UR.ZS":   "wdi_basic_water_urb_pct"        # água potável básica em área urbana
}

def fetch_wdi_indicator(indicator_code: str) -> pd.DataFrame:
    """
    Baixa um indicador do WDI via API do Banco Mundial,
    descompacta o CSV e devolve um DataFrame padronizado
    com as colunas: ['country', 'iso3', 'year', 'value'].
    """
    url = f"https://api.worldbank.org/v2/en/indicator/{indicator_code}?downloadformat=csv"
    print(f"Baixando indicador {indicator_code} ...")
    r = requests.get(url)
    r.raise_for_status()

    # O retorno é um ZIP; vamos abrir em memória
    z = zipfile.ZipFile(io.BytesIO(r.content))
    
    # Procura o CSV principal dentro do ZIP (costuma começar com 'API_')
    csv_name = [name for name in z.namelist() if name.endswith(".csv") and "Metadata" not in name][0]
    with z.open(csv_name) as f:
        df_raw = pd.read_csv(f, skiprows=4)
    
    # Mantém colunas principais: país, código do país e anos
    # O formato é wide (colunas por ano), vamos transformar em long
    id_vars = ["Country Name", "Country Code"]
    value_vars = [c for c in df_raw.columns if c.isdigit()]  # colunas numéricas de ano (ex: '2000', '2001', ...)

    df_long = df_raw.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name="year",
        value_name="value"
    )

    # Limpeza básica
    df_long = df_long.rename(columns={
        "Country Name": "country",
        "Country Code": "iso3"
    })
    df_long["year"] = pd.to_numeric(df_long["year"], errors="coerce")
    df_long = df_long.dropna(subset=["year"])  # remove linhas sem ano
    df_long["year"] = df_long["year"].astype(int)

    # Remove linhas totalmente vazias de valor
    df_long = df_long.dropna(subset=["value"])

    return df_long

# ---------------------------------------------------------
# 1) Baixar todos os indicadores definidos em wdi_indicators
# ---------------------------------------------------------
wdi_frames = []

for code, col_name in wdi_indicators.items():
    df_ind = fetch_wdi_indicator(code)
    df_ind = df_ind.rename(columns={"value": col_name})
    wdi_frames.append(df_ind[["country", "iso3", "year", col_name]])

# ---------------------------------------------------------
# 2) Unir todos em um único DataFrame wide (por iso3 + ano)
# ---------------------------------------------------------
from functools import reduce

# Merge incremental usando iso3 + year
wdi_merged = reduce(
    lambda left, right: pd.merge(
        left, right,
        on=["country", "iso3", "year"],
        how="outer"
    ),
    wdi_frames,
)

print("✅ WDI Urban Core Pack montado:")
print(wdi_merged.head())
print("\nFormato:", wdi_merged.shape)

# ---------------------------------------------------------
# 3) Salvar CSV(s) no diretório `datasets/`
# ---------------------------------------------------------
output_dir = Path('datasets')
output_dir.mkdir(parents=True, exist_ok=True)

wdi_merged_path = output_dir / 'wdi_merged.csv'
wdi_merged.to_csv(wdi_merged_path, index=False)
print(f"Salvo: {wdi_merged_path}")

# (Opcional) salvar cada indicador como CSV individual — útil para inspeção
for col in [v for v in wdi_indicators.values()]:
    # pivot to get iso3, year, value for that indicator only
    subset = wdi_merged[['country', 'iso3', 'year', col]].dropna(subset=[col])
    if not subset.empty:
        path = output_dir / f"{col}.csv"
        subset.to_csv(path, index=False)
        print(f"Salvo: {path}")


Baixando indicador SP.URB.TOTL.IN.ZS ...
Baixando indicador SP.URB.GROW ...
Baixando indicador SP.URB.GROW ...
Baixando indicador EN.ATM.CO2E.PC ...
Baixando indicador EN.ATM.CO2E.PC ...
Baixando indicador EN.ATM.PM25.MC.M3 ...
Baixando indicador EN.ATM.PM25.MC.M3 ...
Baixando indicador EG.FEC.RNEW.ZS ...
Baixando indicador EG.FEC.RNEW.ZS ...
Baixando indicador SH.STA.BASS.UR.ZS ...
Baixando indicador SH.STA.BASS.UR.ZS ...
Baixando indicador SH.H2O.BASW.UR.ZS ...
Baixando indicador SH.H2O.BASW.UR.ZS ...
✅ WDI Urban Core Pack montado:
       country iso3  year  wdi_urb_pop_pct  wdi_urb_pop_growth  \
0  Afghanistan  AFG  1960            8.401                 NaN   
1  Afghanistan  AFG  1961            8.684            5.275389   
2  Afghanistan  AFG  1962            8.976            5.351671   
3  Afghanistan  AFG  1963            9.276            5.392848   
4  Afghanistan  AFG  1964            9.586            5.448598   

   wdi_co2_per_capita  wdi_pm25  wdi_renew_energy_pct  \
0     